## General workflow for writing labbench drivers

### Learn the backend library

Make liberal use of the interpreter to explore the backend library. Find the documentation and make liberal use of `dir` to explore the API exposed by the 
* starting with a tutorial for a the python package directly (pyvisa, pyserial, cdll, win32com, ...), or
* using a bare `labbench` backend implementation like `VISADevice`.

Here is an example that directly uses [pyvisa](pyvisa.readthedocs.io) to control a microwave power sensor:

In [ ]:
import pyvisa
import pandas as pd

rm = pyvisa.ResourceManager('@ni')
inst = rm.open_resource('TCPIP::10.0.0.1::::INSTR')

print(inst.query('INIT:CONT?'))  # Check whether triggering is enabled
print(inst.query('OUTP:TRIG?'))  # Check whether the trigger output pass-through is enabled
print(inst.query('TRIG:SOUR?'))  # Check the flag indicating the trigger source
print(inst.query('TRIG:COUN?'))  # Check the number of triggers to enable
print(inst.query('SENS:MRAT?'))  # Check the measurement speed
print(inst.query('SENS:FREQ?'))  # Check the center frequency setting

response = inst.query('FETC?').split(',')     # Fetch sample(s) and split by comma
if len(response)==1:
    print(float(response[0]))                 # Scalar for single sample response
else:
    print(pd.to_numeric(pd.Series(response))) # 1-dimensional series for vector response

The goal here is to establish the ability to use the backend for connection and communication.

### Identify a base class
`labbench` includes several drivers with support for python modules and other libraries. Each of these backends is accessible in a device instance using the driver `backend` attribute. For example:
```python
device = lb.VISADevice('TCPIP::10.0.0.1')
print(device.backend.query('*IDN?'))
```
connects an instance of a pyvisa resource to the given IP address accesses, and then directly makes a query call using the pyvisa backend. Driver classes should follow this convention the control object in the `backend` attribute this way.

The base `lb.Device` class has the least specific implementation.
Several more specialized subclasses implement popular lab automation protocols:

| Driver class           | Backend module   | property `key` |
|:-----------------------|:-----------|:---------------------|
| lb.ShellBackend     | subprocess | No               |
| lb.Device (base class)    | -          | -                    |
| lb.DotNetDevice           | pythonnet  | No             |
| lb.LabviewSocketInterface | socket     | Yes|
| lb.SerialDevice           | pyserial   | Yes|
| lb.SimulatedVISADevice    | pyvisa       | Yes            |
| lb.VISADevice             | pyvisa     | Yes             |

Selecting one of these as your parent class rather than `Device` gives you
* connection management to establish the `backend` attribute,
* support for the `key` keyword for `lb.property` for automatic parameter implementation of key/value style parameters in the backend library,
* methods and traits for basic use of the backend, and
* some degree of testing and documentation.

### Implementation strategy
The aim of developing a new subclass to control a specific instrument or software should be encapsulate all access to the `backend` attribute needed for lab automation. This way, its usage won't require time spent searching the internet or pdfs for the documentation about the backend. The `Device` object structure includes implementation shortcuts that complement the usual process of implementing methods (functions) in the class. By use case:

* Consider `lb.property` to allow automatic logging of a single metadata or configuration parameter in the `backend`. This fits best for parameters that (1) do not change significantly between calls to get/set, (2) do not require multiple input arguments, and (3) are not slow enough to motivate use of threading. Convention is to name these with noun/object grammar.
* Decorate methods (class functions) with `@lb.datareturn` for read-only operations similar to the `lb.property` use case when multiple (or different) input arguments are needed. Convention is to name these with verb/predicate grammar.
* Implement methods the same was as a normal python class when the above are inappropriate. Some examples could include triggering a sequence of different automation tasks, lengthy measurements, manipulation of complicated data structures, operations that take multiple input arguments. Convention is to name these with verb/predicate grammar.

A setting that controls whether RF output is enabled, for example, could control a boolean as a state, such as 

```python
device.state.rf_output_enabled = True
```
or as a method
```python
device.enable_rf_output(True)
```
If in doubt, try a `lb.property` trait to leverage its casting, validation, and data logging features.

In [1]:
import labbench as lb
import pandas as pd

# Wrap what we learned exploring device into a Device object
# (subclassed as lb.Device->lb.VISADevice->PowerSensor)
class PowerSensor(lb.VISADevice):
    # enumeration constants
    TRIGGER_SOURCES = 'IMM', 'INT','EXT2', 'BUS', 'INT1'
    FUNCTIONS = 'POW:AVG', 'POW:BURS:AVG', 'POW:TSL:AVG', 'XTIM:POW', "XTIM:POWer"
    RATES = 'NORM','DOUB','FAST'

    initiate_continuous = lb.property.bool(key='INIT:CONT')
    output_trigger = lb.property.bool(key='OUTP:TRIG')
    trigger_source = lb.property.str(key='TRIG:SOUR', only=TRIGGER_SOURCES)
    function = lb.property.str(key='SENS:FUNC', only=FUNCTIONS)        
    trigger_count = lb.property.int(key='TRIG:COUN', min=1,max=200)
    measurement_rate = lb.property.str(key='SENS:MRAT', values=RATES)
    sweep_aperture = lb.property.float(key='SWE:APER', min=20e-6, max=200e-3,help='time (s)')
    frequency = lb.property.float(key='SENS:FREQ', min=10e6, max=18e9,help='input center frequency (Hz)')

    @function.setter
    def function(self, value):
        # the write method is provided by the parent class, VISADevice
        self.write(f'SENSe:FUNCtion "{value}"')
        
    @trigger_source.getter
    def trigger_source(self):
        # this instrument accepts 'EXT2' for set, but returns '2' on get
        source = self.query('TRIG:SOUR?')
        if source == '2':
            source = 'EXT2'
        return source
        
    def preset (self):
        # no arguments or return data, and it triggers an action in the device, so it's a method
        self.write('SYST:PRES')

    def fetch (self):
        response = self.query('FETC?').split(',')
        if len(response)==1:
            return float(response[0])
        else:
            return pd.to_numeric(pd.Series(response))

ModuleNotFoundError: No module named 'labbench'

This simple driver hits on all of the basics of implementing a simple device driver using a backend. The most important aspects are discussed below.

#### 1. Every `labbench` driver is a subclass of a labbench Device class, such as lb.VISADevice:

This is the definition of the PowerSensor:
```python
class PowerSensor(lb.VISADevice):
    # ...
```
This single line subclasses (inherits from) the labbench VISADevice class. This means that in this one line, the PowerSensor driver has adopted _all of the same members (methods, state traits, and other attributes) as the supplied "vanilla" VISADevice type_. The `VISADevice` class helps streamline use of the `pyvisa` with features like
* managing connection and disconnection, given the VISA resource string;
* shortcuts for accessing simple instrument states, implemented entirely based on definitions (discussed below); and
* wrapper methods (i.e., member functions) for pyvisa resource `write` and `query` methods.

This power sensor driver definition is just that - a definition. To _use_ the driver and connect to the instrument in the lab, instantiate it and connect to the device. This is the simplest recommended way to instantiate and connect:
```python
# Here is the `with` block
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:
    pass
    # This is where we would do our instrument automation and
    # acquisition with the sensor object
 
# Now the `with` block is done and we're disconnected
print('Disconnected, all done')
```
There are two key pieces here:
* The instantiation, `PowerSensor('TCPIP::10.0.0.1::::INSTR')`, is where we create a power sensor object that we can interact with. All `VISADevice` drivers use this standard resource string formatting; other types of drivers have different formats.
* The `with` block (talked about under the name _context management_ in python language documents) serves two functions for any labbench driver (not just VISADevice):
    1. The instrument is connected at the start of the with block
    2. guarantees that the instrument will be disconnected after the with end of the with block, _even if there is an error inside the block!_
    
#### 2. The `state` object for controlling simple device state parameters
##### 2.1 State trait definition
A each of the attributes defined in the state class is a _descriptor_ (based on [traitlets](https://github.com/ipython/traitlets) under the hood). The example includes seven descriptors:
```python
    class state (lb.VISADevice.state):
        initiate_continuous = lb.Bool              (key='INIT:CONT')
        output_trigger      = lb.Bool              (key='OUTP:TRIG')
        trigger_source      = lb.CaselessStrEnum   (key='TRIG:SOUR',
                                                    values=['IMM','INT','EXT2','BUS','INT1'])
        function            = lb.CaselessStrEnum   (key='SENS:FUNC',
                                                    values=['POW:AVG', 'POW:BURS:AVG',
                                                            'POW:TSL:AVG',
                                                            'XTIM:POW', "XTIM:POWer"])        
        trigger_count       = lb.Int               (key='TRIG:COUN', min=1,max=200)
        measurement_rate    = lb.CaselessStrEnum   (key='SENS:MRAT',
                                                    values=['NORM','DOUB','FAST'])
        sweep_aperture      = lb.Float             (key='SWE:APER',  min=20e-6, max=200e-3,
                                                    help='time (in s)')
        frequency           = lb.Float             (key='SENS:FREQ', min=10e6, max=18e9,
                                                    help='input center frequency (in Hz)')
```
When the `PowerSensor` driver is instantiated, these stop being definitions, and become objects that can be get or set as if they were python variables. Behind the scenes, the `state` object has extra features that can trigger function calls (callbacks) when any state trait changes. This feature can be used to automatically record the changes we make to these states to a database or automatically generate a GUI front-panel.

The definition above includes metadata that dictates the python data type handled for this assignment operation, how it should be converted

| **Type of state trait argument**  | **Examples from `PowerSensor`**  | **Backend-dependent?** 	|
|---------------------------------	|------------------------------------|-------------------	|
| Python data type for assignment 	| `lb.Float`, `lb.CaselessStrEnum` | No   |
| Data validation settings        	| `min`,`max`,`step`   | No    |
|                                   | `values` (for enumerated types) 	 | No     |
| Documentation strings           	| `help`                             | No     |
| Logging tags              	    | `is_metadata`                             | No  |
| Command message      	            | `key='INIT:CONT'`, `key='SENS:MRAT'` | Yes |
(some types of drivers do not use the `command` keyword at all as discussed in [how to write a labbench device driver](how to write a device driver)).

*Every* labbench driver has a state object, including at least the boolean state called `connected` (indicating whether the host computer is connected with the remote device is connected or not). This is inherited from the base `lb.Device` class.

There are two ways to implement a state trait.

###### 2.1.1 Implementing state traits with the command argument
Device communication based on message strings often support a simple syntax for getting or setting state parameters. In these cases, a backend driver like `VISADevice` may implement the `command` argument to streamline implementation of state traits in device drivers.

The `VISADevice` driver tries to use the command argument for each state trait to determine how to communicate with the remote instrument on assignment. In `PowerSensor` above, a get or set the frequency state defined with the command `'SENS:FREQ'` would be implemented by querying or writing the SCPI string `'SENS:FREQ?'` or `'SENS:FREQ 1e9'`.

###### 2.1.2 Implementing a state trait setter manually
Implementing gets and sets of simple traits sometimes requires custom logic. This arises for backend drivers that do not support a simple command structure that can be easily specified with a single `command` argument. Even for devices that do generally support this type of command structure, custom logic is sometimes needed for "quirky" or nonstandard commands. The definition of `PowerSensor` above includes two cases from driver code written for actual commercial power sensors.

```python
    @state.function.setter
    def __ (self, value):
        self.write('SENSe:FUNCtion "{}"'.format(value))
```

VISADevice assumes that the state called "function," defined with
the keyword argument key='SENS:FUNC',
should be set on the remote device using the SCPI string formatted
as 'SENS:FUNC POW:AVG' (where POW:AVG might be replaced by other
enum values in the definition of function above).

This power sensor expects quotes around the parameter supplied to
set SENS:FUNC --- for example 'SENS:FUNC "POW:AVG"'.
This custom method replaces the automatic parameter setting implemented
in VISADevice with this custom behavior.

* The decorator syntax `@state.function.setter` is what connects
  this implementation to state.function.
* This only replaces the VISADevice state *setting* behavior. The
  state *getting* behavior is still the default.
* The name of the method, '__', hides it from users of the class driver;
  it shouldn't need to be called except to implement
  state.function.setter.
* All state setting functions take one argument in addition to self,
  which is the value to send to the device.


###### 2.1.3 Implementing a state trait getter manually
Here is the manually implemented getter method from `PowerSensor`:
```python
    @state.trigger_source.getter
    def __ (self):
        source = self.query('TRIG:SOUR?')
        if source == '2':
            source = 'EXT2'
        return source

```
VISADevice assumes that the state called "trigger_source," defined with
the keyword argument key='TRIG:SOUR',
should be queried on the remote device using the SCPI string formatted
as 'TRIG:SOUR?'. It also expects that the response can be converted
to the specified data type - in this case one of the enumerated strings,
such as 'EXT2'.

Quirks that do not follow this pattern are common for getters (queries),
like the setters above. For example, this power sensor returns
'2' instead of 'EXT2', even though it expects the parameter 'EXT2'
for a set operation. This function implements the special case to
replace '2' (if returned by the device) with 'EXT2' so that there
are no errors if we try to get and set the same value.

This method replaces the automatic parameter setting implemented
in VISADevice with this custom behavior.

* The decorator syntax `@state.trigger_source.getter` is what connects
  this implementation to state.trigger_source.
* This only replaces the VISADevice state *getting* behavior. The
  state *setting* behavior is still the default.
* The name of the method, '__', duplicates the function.setter (above).
  This doesn't matter, since it shouldn't need to be called separately
  by a user anyway.
* These state getting functions take no arguments besides `self`, and return
  the value received from the device.       


##### 2.2 Using state traits to control the device
Here is working example that communicates with the device to get and set state traits. It is implemented entirely with `labbench` internals and the device definition above.

```python
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:

    # This prints True if we're still in the with block
    print(sensor.state.isopen)    
    
    # Use SCPI to request the identity of the sensor,
    # then return and print it. This was inherited from
    # VISADevice, so it is available on any VISADevice driver.
    print(sensor.state.identity)
    
    # PowerSensor.state.frequency is defined as a float. Assigning
    # to it causes logic inherited from lb.VISADevice
    # to convert this to a string, and then write the SCPI string
    # 'SENS:FREQ 2.45e9' to the instrument.
    sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz
    
    # We can also access the remote value of sensor.state.frequency.
    # Behind the scenes, each time we fetch the value, magic in
    # lb.VISADevice retrieves the current value from the instrument
    # with the SCPI query 'SENS:FREQ?', and then converts it to a floating point
    # number.
    print('The sensor frequency is {} GHz'.format(sensor.state.frequency/1e9))
    
print(sensor.state.isopen) # Prints False - we're disconnected
```
Simply put: assigning to or from with the attribute in the driver state instance causes remote set or get operations. The python data type matches the definition in the `state` class.

##### 2.3 Discovering and navigating states
Inheriting from `VISADevice` means that `PowerSensor.state` includes the seven states defined here, plus all others listed provided by VISADevice.state. Since these aren't listed here, it can get confusing tracking what has been inherited (like in other object-oriented libraries). Fortunately, there are many ways to explore the entire list of states that have been inherited from the parent state class:
1. Look it up [in the API reference manual](http://ssm.ipages.nist.gov/labbench/labbench.html#labbench.visa.VISADevice.state)
2. When working with an instantiated driver object in an ipython or jupyter notebook command prompt, type `lb.VISADevice.state.` and press tab to autocomplete a list of valid options. You'll also see some functions to do esoteric things with these states.
3. When working in an editor like pycharm or spyder, you can ctrl+click on the right side of `VISADevice.state` to skip directly to looking at the definition of `VISADevice.state` in the `labbench` library
4. When working in any kind of python prompt, you can use the `help` function
   ```python
   help(PowerSensor.state)
   ```
5. When working in an ipython or jupyter prompt, a nicer option than 4. is the ipython help magick:
   ```python
   PowerSensor.state?
   ```

### 3. Device methods for commands and data acquisition
The `state` class above is useful for remote assignment operations on simple scalar data types. Supporting a broader collection of operation types ("trigger a measurement," "fetch and return measurement data," etc.) need the flexibility of more general-purpose functions. In python, a member function of a class is called a method.

Here are the methods defined in `PowerSensor`:
```python
def preset (self):
    self.write('SYST:PRES')

def fetch (self):
    response = self.query('FETC?').split(',')
    if len(response)==1:
        return float(response[0])
    else:
        return pd.to_numeric(pd.Series(response))
```
These are the methods that are specific to our power sensor device. 
* The `preset` function tells the device to revert to its default state.
* The `fetch` method performs some text processing on the response from the device, and returns either a single scalar or a pandas Series if the result is a sequence of power values.

The `labbench` convention is that the names of these methods are verbs (or sentence predicates, when single words are not specific enough).

##### 3.1 Example data acquisition script
Here is an example that presets the device, sets the center frequency to 2.45 GHz, and then collects 10 power samples:

In [ ]:
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:
    print('Connected to power sensor {}'.format(sensor.state.identity))
    
    sensor.preset()
    sensor.wait()   # VISADevice includes in the standard VISA wait method, which sends SCPI '*WAI'
    sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz

    power_levels = pd.Series([sensor.fetch() for i in range(10)])

print('All done! Got these power levels: ')
print(power_levels) 

##### 3.2 Discovering and navigating device driver methods
Inheritance has similar implications as it does for the `VISADevice.state` class. Inheriting from `VISADevice` means that `PowerSensor` includes the two methods, plus many more from `lb.VISADevice` (some of which it inherited from `lb.Device`). Since these aren't listed in the example definition above, it can get confusing tracking what methods are available through inheritance (like in other object-oriented libraries). Sometimes this confusion is called "abstraction hell." Fortunately, there are many ways to avoid this problem:
1. Look it up [in the API reference manual](http://ssm.ipages.nist.gov/labbench/labbench.html#labbench.visa.VISADevice)
2. When working with an instantiated driver object in an ipython or jupyter notebook command prompt, type `lb.VISADevice.` and press tab to autocomplete a list of valid options. You'll also see some functions to do esoteric things with these states.
3. When working in an editor like pycharm or spyder, you can ctrl+click on the right side of `VISADevice` to skip directly to looking at the definition of `VISADevice` in the `labbench` library
4. When working in any kind of python prompt, you can use the `help` function
   ```python
   help(PowerSensor)
   ```
5. When working in an ipython or jupyter prompt, a nicer option than 4. is the ipython help magick:
   ```python
   PowerSensor?
   ```

## 4 Miscellaneous extras
##### 4.1 Connecting to multiple devices
The best way to connect to multiple devices is to use a single `with` block. For example, a 10-sample acquisition with two power sensors might look like this:

In [ ]:
# The backslash here is **very important**! Without it, you'll get a syntax error.
# (Alternatively, you can put power sensor definitions on the same line, which gets hard to read)
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor1,\
     PowerSensor('TCPIP::10.0.0.2::::INSTR') as sensor2:
    print('Connected to power sensors')
    
    for sensor in sensor1, sensor2:
        sensor.preset()
        sensor.wait()   # VISADevice includes in the standard VISA wait method, which sends SCPI '*WAI'
        sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz
    
    power_levels = pd.DataFrame([[sensor1.fetch(),sensor2.fetch()] for i in range(10)])

print('All done! Got these power levels: ')
print(power_levels) 

##### 4.2 Execute a function on state changes
Database management and user interface tools make extensive use of callbacks, which gives an opportunity for you to execute custom code any time an assignment causes a state to change. A state change can occur in a couple of ways: 
* This triggers a callback if 2.45e9 is different than the last observed frequency:
  ```python
     sensor.state.frequency = 2.45e9
  ```
* This triggers a callback if the instrument returns a frequency that is is different than the last observed frequency
  ```python
     current_freq = sensor.state.frequency 
  ```
  
Configure a function call on an observed change with the `observe` method in `sensor.state`:

In [ ]:
def callback (change):
    """ the callback function is given a single argument. change
        is a dictionary containing the descriptor ('frequency'),
        the state instance that contains frequency, and both
        the old and new values.
    """
    # insert GUI update here?
    # commit updated state to a database here?
    print(change)

with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:
    sensor.state.observe(change)
    
    sensor.preset()
    sensor.wait()   # VISADevice includes in the standard VISA wait method, which sends SCPI '*WAI'
    sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz    

print('All done! Got these power levels: ')
print(power_levels)

Use of callbacks can help separate the actual measurement loop (the contents of the `with` block) from other functions for debugging, GUI, and database management. The result can be code that is more clear.